# Kryptologie Lab - Übung 04 - AES Key Generation

> AES siehe Übung 03!

In [12]:
import numpy as np

## AES aus Übung 03

In [13]:
encryptGaloisMatrix = np.array([[2,3,1,1],[1,2,3,1],[1,1,2,3],[3,1,1,2]])
decryptGaloisMatrix = np.array([[0xE,0xB,0xD,9],[9,0xE,0xB,0xD],[0xD,9,0xE,0xB],[0xB,0xD,9,0xE]])

In [14]:
with open('SBox.txt') as f:
    lines = f.readlines()
    hexValues = "".join(lines).replace("\n", " ").split(", ")
    byteValues = [int(x, base=16) for x in hexValues]
    sbox = np.reshape(np.array(byteValues, dtype=np.dtype('B')), (16,16))

with open('SBoxInvers.txt') as f:
    lines = f.readlines()
    hexValues = "".join(lines).replace("\n", " ").split(", ")
    byteValues = [int(x, base=16) for x in hexValues]
    sbox_inverse = np.reshape(np.array(byteValues, dtype=np.dtype('B')), (16,16))

### Init Galoiskörper-Operationen

In [15]:
def gfAddition(a,b):
    return np.bitwise_xor(a,b)

def xTimes(a):
    t = a << 1
    if a & (1 << 7) != 0:
        t = t ^ 0x1b
    return t

def gfMultiply(a,b):
    index = 0
    out = 0
    while b != 0:
        if (b & 1 == 1):
            val = a
            for i in range(index):
                val = xTimes(val)
            out = gfAddition(out, val)
        index += 1
        b = b >> 1
    return out

### Init algorithm steps

In [16]:
def initBlock(textblock):
    chars = [ord(c) for c in textblock]
    if (len(chars) < 16):
        fill = [0 for i in range(16 - len(chars))]
        chars.extend(fill)
    return np.reshape(np.array(chars, dtype=np.dtype('B')), (4,4), order="F")

def blockToText(textblock):
    chars = [chr(c) for c in np.nditer(textblock, order="F")]
    return "".join(chars)

def addRoundKey(textblock, key):
    return np.bitwise_xor(textblock, key)
    
def subByte(byte, box):
    row = (byte >> 4) & 0xF
    col = byte & 0xF
    return box[row, col]

def subBytes(textblock, reverse=False):
    if reverse:
        box = sbox_inverse
    else:
        box = sbox
    for y in range(len(textblock)):
        for x in range(len(textblock[y])):
            textblock[y,x] = subByte(textblock[y,x], box)
    return textblock

def shiftRows(textblock, reverse=False):
    for i in range(4):
        if reverse:
            shiftAmount = i
        else:
            shiftAmount = -i
        textblock[i] = np.roll(textblock[i], shiftAmount)
    return textblock

def subColumn(inputColumn, matrix):
    outputColumn = np.copy(inputColumn)
    for row in range(4):
        val = 0
        for column in range(4):
            val = gfAddition(val, gfMultiply(matrix[row, column], inputColumn[column]))
        outputColumn[row] = val
    return outputColumn

def mixColumns(textblock, reverse=False):
    if reverse:
        matrix = decryptGaloisMatrix
    else:
        matrix = encryptGaloisMatrix
    for column in range(4):
        textblock[:, column] = subColumn(textblock[:, column], matrix)
    return textblock

### Init algorithm process

In [17]:
def encryptBlock(textblock, keys):
    textblock = initBlock(textblock)
    textblock = addRoundKey(textblock, keys[0])
    for i in range(1, 10):
        textblock = subBytes(textblock)
        textblock = shiftRows(textblock)
        textblock = mixColumns(textblock)
        textblock = addRoundKey(textblock, keys[i])
    textblock = subBytes(textblock)
    textblock = shiftRows(textblock)
    textblock = addRoundKey(textblock, keys[10])
    return blockToText(textblock)

def decryptBlock(textblock, keys):
    textblock = initBlock(textblock)
    textblock = addRoundKey(textblock, keys[10])
    textblock = shiftRows(textblock, reverse=True)
    textblock = subBytes(textblock, reverse=True)
    for i in range(9, 0, -1):
        textblock = addRoundKey(textblock, keys[i])
        textblock = mixColumns(textblock, reverse=True)
        textblock = shiftRows(textblock, reverse=True)
        textblock = subBytes(textblock, reverse=True)
    textblock = addRoundKey(textblock, keys[0])
    return blockToText(textblock)

### Init block splitting

In [18]:
def encrypt(text, keys):
    blocks = [text[i:i+16] for i in range(0, len(text), 16)]
    return "".join([encryptBlock(block, keys) for block in blocks])

def decrypt(text, keys):
    blocks = [text[i:i+16] for i in range(0, len(text), 16)]
    return "".join([decryptBlock(block, keys) for block in blocks])

## Schlüsselgenerierung

In [19]:
# RotWord(W) = RotWord(b0, b1, b2, b3) = (b1, b2, b3, b0)
def rotWord(word):
    return np.roll(word, shift=-1)

# SubWord(b0, b1, b2, b3) = (S[b0], S[b1], S[b2], S[b3])
# subByte from AES (SBox)
def subWord(word):
    return [subByte(byte, sbox) for byte in word]

# rcon(i) = (rci 0016 0016 0016) 
# rci = x^(i-1) in GF -> implemented as lookup table
def rcon(i):
    array = [0x1, 0x2, 0x4, 0x8, 0x10, 0x20, 0x40, 0x80, 0x1b, 0x36]
    return [array[i], 0,0,0]

# input 16 character string
def generateKey(key):
    block = initBlock(key)
    words = []
    # first 4 words are directly from the key
    words.append(block[:,0])
    words.append(block[:,1])
    words.append(block[:,2])
    words.append(block[:,3])
    # generates 44 words
    for i in range(4, 44):
        if i % 4 == 0:
            val = gfAddition(words[i-4], rcon(i // 4 - 1))
            val = gfAddition(subWord(rotWord(words[i-1])), val)
            words.append(val)
        else:
            words.append(gfAddition(words[i-4], words[i-1]))
    keys = np.empty((11, 4,4), dtype=np.dtype('B'))
    for i in range(0, 44):
        keys[i // 4, :, i % 4] = words[i]
    return keys

In [20]:
keys = generateKey("1234567890123456")
print(keys)
print(keys.shape)

[[[ 49  53  57  51]
  [ 50  54  48  52]
  [ 51  55  49  53]
  [ 52  56  50  54]]

 [[ 40  29  36  23]
  [164 146 162 150]
  [ 54   1  48   5]
  [247 207 253 203]]

 [[186 167 131 148]
  [207  93 255 105]
  [ 41  40  24  29]
  [  7 200  53 254]]

 [[ 71 224  99 247]
  [107  54 201 160]
  [146 186 162 191]
  [ 37 237 216  38]]

 [[175  79  44 219]
  [ 99  85 156  60]
  [101 223 125 194]
  [ 77 160 120  94]]

 [[ 84  27  55 236]
  [ 70  19 143 179]
  [ 61 226 159  93]
  [244  84  44 114]]

 [[ 25   2  53 217]
  [ 10  25 150  37]
  [125 159   0  93]
  [ 58 110  66  48]]

 [[102 100  81 136]
  [ 70  95 201 236]
  [121 230 230 187]
  [ 15  97  35  19]]

 [[ 40  76  29 149]
  [172 243  58 214]
  [  4 226   4 191]
  [203 170 137 154]]

 [[197 137 148   1]
  [164  87 109 187]
  [188  94  90 229]
  [225  75 194  88]]

 [[ 25 144   4   5]
  [125  42  71 252]
  [214 136 210  55]
  [157 214  20  76]]]
(11, 4, 4)


In [21]:
text = "Hallo, das ist ein geheimer Text. Mal schauen, ob er geheim bleibt. Der Text ist noch ein Stueck laenger."
print("Input: ", text)
cipher = encrypt(text, keys)
print("cypher: ", cipher)
new_text = decrypt(cipher, keys)
print("text: ", new_text)

Input:  Hallo, das ist ein geheimer Text. Mal schauen, ob er geheim bleibt. Der Text ist noch ein Stueck laenger.
cypher:  T ~û9};eJÛZuÔVU0Þö%U{ë$­tÚÓ*%/þã_Å­Aÿ¨¯svRiãÌø})®ä<ªÊ¿Îy£nÝâaÏ@&){|éaçz!¦Õ:¥G×Û~óªOÈ
text:  Hallo, das ist ein geheimer Text. Mal schauen, ob er geheim bleibt. Der Text ist noch ein Stueck laenger.       
